IMPORTING ALL LIBRARIES

In [11]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import requests
from io import StringIO
from io import BytesIO


AUTHENTICATION SETUP

This part authenticates the Python script with Google APIs

In [12]:
SERVICE_ACCOUNT_FILE = '/content/projectapi-446706-a758c3c45fdc.json'
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

#Path to your service account's JSON credentials file. This file contains details like private_key, client_email, and project_id for authentication
#scope script defines the level of access (permissions) that your script is requesting from the Google API.  (Here the scope is read only.)


CREATE THE GOOGLE DRIVE SERVICE

This step sets up a connection to Google Drive.

In [13]:
Credentials= service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)   #Loads the credentials from the JSON file.
service= build('drive','v3',credentials= Credentials)
#This line builds the Google Drive API service, allowing your script to interact with Google Drive. Here's what it does
#build() is a function provided by the googleapiclient.discovery module that initializes the API service
#The first argument, 'drive', specifies the Google Drive API you want to use.
#The v3 is the API version.

FOLDER ID

Storing the Drive folder Id


In [14]:
Folder_ID= '147geiTgBKpYfYhpq0tiAkJcjhnvxu6w9'

FUNCTION TO FET THE FILE AND STORE IT IN LIST

In [17]:
def fetch_files_from_drive(service, folder_id):
    results = service.files().list(q=f"'{folder_id}' in parents", fields="files(id, name, mimeType)").execute()
    file_dataframes = []

    for file in results.get('files', []):
        file_id, mime_type = file['id'], file['mimeType']
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
        response = requests.get(download_url)

        if response.status_code == 200:
            if mime_type == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
                df = pd.read_excel(BytesIO(response.content))
            else:  # For CSV or Google Sheets (exported as CSV)
                df = pd.read_csv(StringIO(response.content.decode('utf-8')))
            file_dataframes.append(df)

    return pd.concat(file_dataframes, ignore_index=True) if file_dataframes else None

# Fetch and combine files
combined_df = fetch_files_from_drive(service, Folder_ID)

In [20]:
pd.set_option('display.max_columns', None)
combined_df

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x18d02,CUS_0x9ca,January,Vellacottf,38,401-46-8317,Mechanic,60530.7,4962.225,3,6,9,-100,Auto Loan,24,19,8.35,2.0,Standard,361.83,39.069245,NaN,No,40.763173,646.2799895980319,Low_spent_Small_value_payments,99.179338,Standard
1,0x18d03,CUS_0x9ca,February,Vellacottf,8523_,401-46-8317,Mechanic,60530.7,4962.225,3,6,9,1,Auto Loan,24,16,3.35,2.0,Standard,361.83,25.572516,28 Years and 11 Months,No,40.763173,455.61391483579087,Low_spent_Medium_value_payments,279.845413,Standard
2,0x18d04,CUS_0x9ca,March,NaN,38,401-46-8317,Mechanic,60530.7,4962.225,3,6,9,1,Auto Loan,24,19,8.35,7.0,Standard,361.83,28.985767,29 Years and 0 Months,No,40.763173,225.3827659701968,Low_spent_Medium_value_payments,510.076561,Standard
3,0x18d05,CUS_0x9ca,April,Vellacottf,38,401-46-8317,Mechanic,60530.7,4962.225,3,6,9,1,Auto Loan,24,19,8.35,7.0,Standard,361.83,26.210677,29 Years and 1 Months,NM,40.763173,75.93221359716077,High_spent_Large_value_payments,619.527114,Standard
4,0x18d06,CUS_0x9ca,May,Vellacottf,38,401-46-8317,Mechanic,60530.7,4962.225,3,6,9,1,Auto Loan,25,22,8.35,7.0,Standard,361.83,39.427326,29 Years and 2 Months,NM,40.763173,96.25115895153552,High_spent_Medium_value_payments,609.208168,Standard
